In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark.stop()

In [3]:
from pyspark.sql import SparkSession
mysql_connector_path = "/home/hadoop/Downloads/mysql-connector-java-8.0.13.jar"

In [4]:
spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()

In [ ]:
#pyspark --jars "path to jar"

In [5]:
hremployeeDF = spark.read.format("jdbc")\
.option("url", "jdbc:mysql://localhost:3306/hremployeeDB")\
.option("dbtable", "hremployee").option("user", "root").option("password", "hadoop@123")\
.option("driver", "com.mysql.cj.jdbc.Driver").load()

In [6]:
hremployeeDF.show()

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+--

In [8]:
hremployeeDF.createOrReplaceTempView('hremployee')

In [9]:
# show physical plan of execution
hremployeeDF.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(hremployee) [numPartitions=1] [EmployeeID#0,Department#1,JobRole#2,Attrition#3,Gender#4,Age#5,MaritalStatus#6,Education#7,EducationField#8,BusinessTravel#9,JobInvolvement#10,JobLevel#11,JobSatisfaction#12,Hourlyrate#13,Income#14,Salaryhike#15,OverTime#16,Workex#17,YearsSinceLastPromotion#18,EmpSatisfaction#19,TrainingTimesLastYear#20,WorkLifeBalance#21,Performance_Rating#22] PushedFilters: [], ReadSchema: struct<EmployeeID:string,Department:string,JobRole:string,Attrition:string,Gender:string,Age:int,...


### 1. Display shape of hremployee table
    * show number of records and number of columns

In [10]:
hremployeeDF.count()

1469

In [11]:
spark.sql("DESCRIBE hremployee").select("col_name").count()

23

In [12]:
# write a query to show first 3 employees from each job role
hremployeeDF.printSchema()

root
 |-- EmployeeID: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- JobInvolvement: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: string (nullable = true)
 |-- Hourlyrate: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Salaryhike: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Workex: integer (nullable = true)
 |-- YearsSinceLastPromotion: integer (nullable = true)
 |-- EmpSatisfaction: string (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: string (nullable = true)
 |-- Performance_Rating: string (null

In [13]:
spark.sql("""SELECT EmployeeID,Department,JobRole,row_number() over(partition by JobRole order by EmployeeID) as row_num from hremployee""").show()

+----------+----------+---------------+-------+
|EmployeeID|Department|        JobRole|row_num|
+----------+----------+---------------+-------+
|         1|     Sales|Sales Executive|      1|
|      1012|     Sales|Sales Executive|      2|
|      1014|     Sales|Sales Executive|      3|
|      1020|     Sales|Sales Executive|      4|
|      1026|     Sales|Sales Executive|      5|
|      1027|     Sales|Sales Executive|      6|
|      1031|     Sales|Sales Executive|      7|
|      1032|     Sales|Sales Executive|      8|
|      1039|     Sales|Sales Executive|      9|
|      1042|     Sales|Sales Executive|     10|
|      1048|     Sales|Sales Executive|     11|
|      1049|     Sales|Sales Executive|     12|
|      1050|     Sales|Sales Executive|     13|
|      1052|     Sales|Sales Executive|     14|
|      1058|     Sales|Sales Executive|     15|
|      1059|     Sales|Sales Executive|     16|
|      1063|     Sales|Sales Executive|     17|
|      1067|     Sales|Sales Executive| 

In [14]:
spark.sql("""SELECT * from (select EmployeeID,Department,JobRole,row_number() 
over(partition by JobRole order by EmployeeID) as row_num from hremployee)
where row_num < 4""").show()

+----------+--------------------+--------------------+-------+
|EmployeeID|          Department|             JobRole|row_num|
+----------+--------------------+--------------------+-------+
|         1|               Sales|     Sales Executive|      1|
|      1012|               Sales|     Sales Executive|      2|
|      1014|               Sales|     Sales Executive|      3|
|      1003|Research & Develo...|Manufacturing Dir...|      1|
|      1034|Research & Develo...|Manufacturing Dir...|      2|
|      1038|Research & Develo...|Manufacturing Dir...|      3|
|       100|Research & Develo...|Laboratory Techni...|      1|
|      1001|Research & Develo...|Laboratory Techni...|      2|
|      1002|Research & Develo...|Laboratory Techni...|      3|
|      1013|               Sales|Sales Representative|      1|
|      1022|               Sales|Sales Representative|      2|
|      1057|               Sales|Sales Representative|      3|
|        10|Research & Develo...|Healthcare Repres...| 

In [20]:
### 3 write a query to show top 3 employee from each job role earning max salary
spark.sql("""
  SELECT * FROM (
    SELECT EmployeeID, Department, JobRole, Income,
           rank() OVER (PARTITION BY JobRole ORDER BY Income DESC) AS rank
    FROM hremployee
  ) WHERE rank < 4
""").show()

+----------+--------------------+--------------------+------+----+
|EmployeeID|          Department|             JobRole|Income|rank|
+----------+--------------------+--------------------+------+----+
|        99|               Sales|     Sales Executive| 13872|   1|
|       545|               Sales|     Sales Executive| 13770|   2|
|       839|               Sales|     Sales Executive| 13758|   3|
|       722|Research & Develo...|Manufacturing Dir...| 13973|   1|
|       628|Research & Develo...|Manufacturing Dir...| 13826|   2|
|       744|Research & Develo...|Manufacturing Dir...| 13726|   3|
|       678|Research & Develo...|Laboratory Techni...|  7403|   1|
|       817|Research & Develo...|Laboratory Techni...|  6782|   2|
|       945|Research & Develo...|Laboratory Techni...|  6674|   3|
|       565|               Sales|Sales Representative|  6632|   1|
|      1308|               Sales|Sales Representative|  5405|   2|
|      1220|               Sales|Sales Representative|  4502| 

In [24]:
# 4 show top 3 highest package from overall job role
spark.sql("""
  SELECT JobRole, Income
  FROM (
    SELECT JobRole, Income,
           rank() OVER (ORDER BY Income DESC) AS rank
    FROM hremployee
  )WHERE rank <= 3
""").show()

+-----------------+------+
|          JobRole|Income|
+-----------------+------+
|          Manager| 19999|
|Research Director| 19973|
|          Manager| 19943|
+-----------------+------+



In [29]:
spark.sql("""SELECT EmployeeID, Department, JobRole, Income,
           rank() OVER (PARTITION BY JobRole ORDER BY Income DESC) AS rank
    FROM hremployee""").show()

+----------+----------+---------------+------+----+
|EmployeeID|Department|        JobRole|Income|rank|
+----------+----------+---------------+------+----+
|        99|     Sales|Sales Executive| 13872|   1|
|       545|     Sales|Sales Executive| 13770|   2|
|       839|     Sales|Sales Executive| 13758|   3|
|       976|     Sales|Sales Executive| 13695|   4|
|       296|     Sales|Sales Executive| 13525|   5|
|      1403|     Sales|Sales Executive| 13341|   6|
|       751|     Sales|Sales Executive| 13320|   7|
|      1327|     Sales|Sales Executive| 13225|   8|
|       402|     Sales|Sales Executive| 13212|   9|
|       707|     Sales|Sales Executive| 13194|  10|
|       822|     Sales|Sales Executive| 13120|  11|
|      1223|     Sales|Sales Executive| 12936|  12|
|      1462|     Sales|Sales Executive| 12031|  13|
|       647|     Sales|Sales Executive| 11836|  14|
|       625|     Sales|Sales Executive| 10934|  15|
|       459|     Sales|Sales Executive| 10932|  16|
|       159|

### dense rank gives same value to two same rows

In [37]:

spark.sql("""
select employeeid, department, jobrole, age, gender, income, workex,dense_rank() 
over(partition by JobRole order by age desc, workex desc) as Rank_By_Age from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|Rank_By_Age|
+----------+----------+---------------+---+------+------+------+-----------+
|       428|     Sales|Sales Executive| 60|Female| 10266|    22|          1|
|       880|     Sales|Sales Executive| 60|  Male|  5220|    12|          2|
|       537|     Sales|Sales Executive| 60|  Male|  5405|    10|          3|
|        64|     Sales|Sales Executive| 59|Female|  7637|    28|          4|
|        71|     Sales|Sales Executive| 59|Female|  5473|    20|          5|
|       898|     Sales|Sales Executive| 59|Female|  5171|    13|          6|
|        99|     Sales|Sales Executive| 58|  Male| 13872|    38|          7|
|       402|     Sales|Sales Executive| 56|Female| 13212|    36|          8|
|      1371|     Sales|Sales Executive| 56|Female|  5380|     6|          9|
|        83|     Sales|Sales Executive| 55|  Male| 10239|    24|         10|

In [38]:
spark.sql("""
select employeeid, department, jobrole, age, gender, income, workex,dense_rank() 
over(partition by JobRole order by age , workex desc) as Rank_By_Age from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|Rank_By_Age|
+----------+----------+---------------+---+------+------+------+-----------+
|       381|     Sales|Sales Executive| 24|Female|  4260|     5|          1|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|          1|
|        97|     Sales|Sales Executive| 24|Female|  4999|     4|          2|
|       526|     Sales|Sales Executive| 24|Female|  4577|     4|          2|
|       108|     Sales|Sales Executive| 25|  Male|  5744|     6|          3|
|       139|     Sales|Sales Executive| 25|  Male|  8639|     6|          3|
|       564|     Sales|Sales Executive| 25|Female|  6180|     6|          3|
|       994|     Sales|Sales Executive| 25|  Male|  6232|     6|          3|
|      1433|     Sales|Sales Executive| 25|Female|  4907|     6|          3|
|       398|     Sales|Sales Executive| 25|Female|  4487|     5|          4|

### lag window function
    *allows you to  access records from previous row
    *used in comparing records between rows or calculating differences
    *takes 3 values column name,offset & default 

In [39]:
spark.sql("""
select employeeid, department, jobrole, age, gender, income, workex,lag(income) 
over(partition by JobRole order by employeeid) as previous_income from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+---------------+
|employeeid|department|        jobrole|age|gender|income|workex|previous_income|
+----------+----------+---------------+---+------+------+------+---------------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|           null|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|           5993|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|           9278|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|           4779|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|           5647|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|           4162|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|           9204|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|          10793|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|          10096|
|      1042|     Sales|Sales

In [41]:
spark.sql("""
select employeeid, department, jobrole, age, gender, income, workex,lag(income,3,0) 
over(partition by JobRole order by employeeid) as previous_income from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+---------------+
|employeeid|department|        jobrole|age|gender|income|workex|previous_income|
+----------+----------+---------------+---+------+------+------+---------------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|              0|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|              0|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|              0|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|           5993|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|           9278|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|           4779|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|           5647|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|           4162|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|           9204|
|      1042|     Sales|Sales

### lead function 
    * next value

In [42]:
spark.sql("""
select employeeid, department, jobrole, age, gender, income, workex,lead(income,1,0) 
over(partition by JobRole order by employeeid) as next_income from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|next_income|
+----------+----------+---------------+---+------+------+------+-----------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|       9278|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|       4779|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|       5647|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|       4162|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|       9204|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|      10793|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|      10096|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|       5486|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|       8463|
|      1042|     Sales|Sales Executive| 28|  Male|  8463|     6|       4373|

### NTILE(): 
    *used to find quantiles AND divinding records into quantiles

In [47]:
spark.sql("""
select employeeid, department, jobrole, income, workex, NTILE(4) 
over(order by income desc) as salary_quantile from hremployee
""").show(500)

+----------+--------------------+--------------------+------+------+---------------+
|employeeid|          department|             jobrole|income|workex|salary_quantile|
+----------+--------------------+--------------------+------+------+---------------+
|       191|Research & Develo...|             Manager| 19999|    34|              1|
|       747|Research & Develo...|   Research Director| 19973|    21|              1|
|       852|Research & Develo...|             Manager| 19943|    28|              1|
|       166|Research & Develo...|             Manager| 19926|    21|              1|
|       569|Research & Develo...|             Manager| 19859|    24|              1|
|       919|               Sales|             Manager| 19847|    31|              1|
|       750|               Sales|             Manager| 19845|    33|              1|
|      1242|               Sales|             Manager| 19833|    21|              1|
|       899|Research & Develo...|   Research Director| 19740|    

In [51]:
# find out the mean salary median and max salary for each quantile
spark.sql("""select salary_quantile,avg(income) as mean,percentile(income,0.5) as median , max(income) as max_salary 
from(select employeeid, department, jobrole, income, workex, NTILE(4)
over(order by income) as salary_quantile from hremployee) 
group by salary_quantile
""").show()

+---------------+------------------+-------+----------+
|salary_quantile|              mean| median|max_salary|
+---------------+------------------+-------+----------+
|              1|2353.9972826086955| 2378.0|      2911|
|              2|3966.7356948228885| 4033.0|      4930|
|              3| 6191.138964577656| 6091.0|      8380|
|              4|13522.326975476839|13120.0|     19999|
+---------------+------------------+-------+----------+



### percent rank
    * gives distributed probability as response it calculates the relative rank of row within a result set expressed as percentage

In [54]:
spark.sql("""
select employeeid, department, jobrole, income, workex, PERCENT_RANK()
over(partition by department order by income desc) as perc_rank from hremployee
""").show()

+----------+----------+-------+------+------+--------------------+
|employeeid|department|jobrole|income|workex|           perc_rank|
+----------+----------+-------+------+------+--------------------+
|       919|     Sales|Manager| 19847|    31|                 0.0|
|       750|     Sales|Manager| 19845|    33|0.002252252252252...|
|      1242|     Sales|Manager| 19833|    21|0.004504504504504...|
|      1116|     Sales|Manager| 19586|    36|0.006756756756756757|
|       234|     Sales|Manager| 19517|    32|0.009009009009009009|
|      1126|     Sales|Manager| 19331|    27| 0.01126126126126126|
|       238|     Sales|Manager| 19068|    33|0.013513513513513514|
|        30|     Sales|Manager| 18947|    22|0.015765765765765764|
|       914|     Sales|Manager| 18824|    26|0.018018018018018018|
|       917|     Sales|Manager| 18789|    26| 0.02027027027027027|
|       742|     Sales|Manager| 18303|    21| 0.02252252252252252|
|       908|     Sales|Manager| 18213|    26|0.024774774774774

In [60]:
spark.sql("""
  SELECT 
    CASE 
      when perc_rank <= 0.25 then '0-25'
      when perc_rank <= 0.50 then '25-50'
      when perc_rank <= 0.75 then '50-75'
      else '75-100'
    END AS percentile_range, 
    COUNT(*) AS count_empl
  FROM (
    SELECT 
      employeeid, department, jobrole, income, workex, 
      PERCENT_RANK() OVER(PARTITION BY department ORDER BY income DESC) AS perc_rank
    FROM hremployee
  )
  GROUP BY percentile_range
  ORDER BY percentile_range
""").show()

+----------------+----------+
|percentile_range|count_empl|
+----------------+----------+
|            0-25|       369|
|           25-50|       367|
|           50-75|       366|
|          75-100|       367|
+----------------+----------+



In [72]:
#show job role with highest attrition rate in percentage
spark.sql("""
select jobrole, round(sum(case when Attrition = 'Yes' then 1 else 0 end)*100/(count(*)),3) 
as attrition_rate from hremployee
group by jobrole
order by attrition_rate desc
""").show()


+--------------------+--------------+
|             jobrole|attrition_rate|
+--------------------+--------------+
|Sales Representative|        40.244|
|Laboratory Techni...|        23.938|
|     Human Resources|        23.077|
|     Sales Executive|        17.485|
|  Research Scientist|        16.096|
|Manufacturing Dir...|         6.897|
|Healthcare Repres...|          6.87|
|             Manager|         4.902|
|   Research Director|           2.5|
+--------------------+--------------+



In [1]:
spark.stop()

### hive integration with pyspark

In [2]:
# Spark Integration with Hive with SPARK Session
spark = (SparkSession.builder.appName("pyspark-hive-integration")
        .config('spark.sql.warehouse.dir','/user/hive/warehouse/')
        .enableHiveSupport().getOrCreate())

In [3]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|  banking_db|
|     default|
+------------+



In [5]:
spark.sql("create database if not exists airlines ")

DataFrame[]

In [6]:
spark.sql("""create table if not exists airlines. flights (DayofMonth int, DayOfWeek int, 
Carrier varchar(10), OriginAirportID int, DestAirportID int, DepDelay int,
ArrDelay int)
row format delimited
fields terminated by ','
lines terminated by '\n'
STORED AS TEXTFILE
tblProperties("skip.header.line.count" = "1")""")

DataFrame[]

In [7]:
spark.sql("use airlines")

DataFrame[]

In [8]:
spark.sql("describe flights").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     DayofMonth|      int|   null|
|      DayOfWeek|      int|   null|
|        Carrier|   string|   null|
|OriginAirportID|      int|   null|
|  DestAirportID|      int|   null|
|       DepDelay|      int|   null|
|       ArrDelay|      int|   null|
+---------------+---------+-------+



In [9]:
spark.sql("""load data local inpath '/home/hadoop/Downloads/raw_flight_data1.csv' 
overwrite into table flights""")

DataFrame[]

In [10]:
spark.sql("select * from flights limit 3").show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
+----------+---------+-------+---------------+-------------+--------+--------+



In [11]:
spark.sql("""create table if not exists airlines. airport (airport_id int,city varchar(10),
state varchar(10),name varchar(30))
row format delimited
fields terminated by ','
lines terminated by '\n'
STORED AS TEXTFILE
tblProperties("skip.header.line.count" = "1")""")

DataFrame[]

In [12]:
spark.sql("use airlines")

DataFrame[]

In [15]:
spark.sql("describe airport").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|airport_id|      int|   null|
|      city|   string|   null|
|     state|   string|   null|
|      name|   string|   null|
+----------+---------+-------+



In [16]:
spark.sql("""load data local inpath '/home/hadoop/Downloads/airports1.csv' 
overwrite into table airport""")

DataFrame[]

In [17]:
spark.sql("select * from airport limit 3").show()

+----------+----------+-----+--------------------+
|airport_id|      city|state|                name|
+----------+----------+-----+--------------------+
|     10165|Adak Islan|   AK|                Adak|
|     10299| Anchorage|   AK|Ted Stevens Ancho...|
|     10304|     Aniak|   AK|       Aniak Airport|
+----------+----------+-----+--------------------+



In [18]:
flights_df = spark.table("flights")

In [21]:
airports_df = spark.table("airport")

In [22]:
airports_df.show(2)

+----------+----------+-----+--------------------+
|airport_id|      city|state|                name|
+----------+----------+-----+--------------------+
|     10165|Adak Islan|   AK|                Adak|
|     10299| Anchorage|   AK|Ted Stevens Ancho...|
+----------+----------+-----+--------------------+
only showing top 2 rows



### Transformation

In [32]:
flights_join = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='inner')

In [33]:
flights_join.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|      city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|   Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake |   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|  Portland|   OR|Portland Internat...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
only showing top 3 rows



In [26]:
flights_join = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='left')

In [27]:
flights_join.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|      city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|   Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake |   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|  Portland|   OR|Portland Internat...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+
only showing top 3 rows



In [28]:
flights_join = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='right')

In [29]:
flights_join.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|        19|        5|     DL|          14570|        14869|      -5|      -8|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        19|        5|     DL|          14570|        14869|      -4|       2|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        21|        7|     DL|          14570|        14869|      -2|     -10|     14570|Reno|   NV|Reno/Tahoe Intern...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
only showing top 3 rows



In [65]:
flights_joino = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='outer')

In [67]:
flights_joino.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|        19|        5|     DL|          14570|        14869|      -5|      -8|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        19|        5|     DL|          14570|        14869|      -4|       2|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        21|        7|     DL|          14570|        14869|      -2|     -10|     14570|Reno|   NV|Reno/Tahoe Intern...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
only showing top 3 rows



In [59]:
flights_join1 = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='leftsemi')

In [60]:
flights_join1.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [55]:
flights_join1 = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='leftanti')

In [56]:
flights_join1.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
+----------+---------+-------+---------------+-------------+--------+--------+



In [52]:
flights_join1 = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID,how='leftsemi')

In [53]:
flights_join1.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 3 rows



In [71]:
flights_joino.write.partitionBy('Carrier').parquet("/airlines")

In [72]:
# to read
airlines_join = spark.read.parquet("/airlines")

In [73]:
airlines_join.show(3)

+----------+---------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+-------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|      city|state|                name|Carrier|
+----------+---------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+-------+
|        17|        3|          11298|        11278|      59|      63|     11298|Dallas/For|   TX|Dallas/Fort Worth...|     AA|
|        18|        4|          11298|        11278|       3|      -7|     11298|Dallas/For|   TX|Dallas/Fort Worth...|     AA|
|        19|        5|          11298|        11278|      -3|       0|     11298|Dallas/For|   TX|Dallas/Fort Worth...|     AA|
+----------+---------+---------------+-------------+--------+--------+----------+----------+-----+--------------------+-------+
only showing top 3 rows



In [74]:
airlines_join.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Carrier: string (nullable = true)



In [75]:
airlines_join.createOrReplaceTempView('airlines')

In [76]:
spark.sql("select avg(ArrDelay) as arrival_Delay from airlines where Carrier='AA'").show()

+------------------+
|     arrival_Delay|
+------------------+
|11.651327332054372|
+------------------+



#### load on mysql

In [77]:
connection_prop = {
'user': "root",
'password': 'hadoop@123',
'driver':'com.mysql.cj.jdbc.Driver'
} 
airlines_join.write.jdbc(url = 'jdbc:mysql://localhost:3306/airlines', table = 'airline_join', 
                         mode 'overwrite', properties = connection_prop)

SyntaxError: invalid syntax (<ipython-input-77-4c49158d16c7>, line 7)

#### HBase Integration with Pyspark

In [1]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HbaseIntegration").getOrCreate()

In [3]:
hbase_conf = {
    "hbase.zookeper.quorum":'localhost',
    "hbase.mapreduce.inputtable":'hcustomer',
    "hbase.client.scanner.timeout.period":"600000"
}

In [4]:
df = spark.read.format("org.apache.hadoop.hbase.spark").options(**hbase_conf).load()

Py4JJavaError: An error occurred while calling o67.load.
: java.lang.ClassNotFoundException: Failed to find data source: org.apache.hadoop.hbase.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:213)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.